# NOAA
- request example: https://www.ncei.noaa.gov/access/services/data/v1?dataset=local-climatological-data&dataTypes=HourlyWindSpeed&dataTypes=HourlyDryBulbTemperature&stations=72530094846&startDate=2018-05-01&endDate=2018-05-31
- list of stationsids: https://www.itl.nist.gov/div898/winds/asos-wx/WBAN-MSC.TXT
 - list of WBAN and lat/long: https://www.epa.gov/sites/default/files/documents/STATION_LOCATIONS.PDF
- Other Notes: https://www.ncei.noaa.gov/access/services/support/v3/datasets.json

- Fort Bend WBAN: 12977

![Alt text](image.png)

In [15]:
#needed to make web requests
import requests
#store the data we get as a dataframe
import pandas as pd
#for storing and pulling data
import bp_sql as bp
#mathematical operations on lists
import numpy as np
#parse the datetimes we get from NOAA
import datetime
import os
import sqlite3

## Setup

### Read in Station Names

In [17]:
tday = datetime.datetime.today().replace(hour=0,minute=0,second=0,microsecond=0)
tday_str = tday.strftime('%m-%d-%Y')

db_name = 'weather.db'
conn = bp.create_connection(db_name)

cwd = os.getcwd()

download_fldr = os.path.join(cwd, 'downloads')


sql_create_awsmcs_wban = ''' Create Table if not exists wban_stations (
                                    AWSMSC text,
                                    WBAN text,
                                    NAME text,
                                    AWSMSC_WBAN text
                                    );
                                '''


bp.create_table(db_name, sql_create_awsmcs_wban)


# pull from database table or else repull from website and load into db
try:
    awsmcs_wban_df = pd.read_sql_query(con=conn, sql='Select* from wban_stations')

except sqlite3.OperationalError:


    # sites
    stations_link = 'https://www.itl.nist.gov/div898/winds/asos-wx/WBAN-MSC.TXT'

    # read in html byte
    i = requests.get(stations_link)
    content = i.content
    decoded_content  = str(content,'UTF-8')

    end_list = []

    #skip first 6 rows since it doesn't contain actual data
    #read cols in first then create list with all data
    #Couldnt pd.read_csv bc some of the names are 3+ words and it would skip the >2+ NAMEs
    for n,i in enumerate(decoded_content.splitlines()[6:]):
        if n ==0:
            cols = list(i.split())
        else:
            nums = i.split()[0:2]
            names = " ".join(i for i in i.split()[2:])
            
            nums.append(names)

            end_list.append(nums)

    awsmcs_wban_df = pd.DataFrame(end_list, columns=cols)
    awsmcs_wban_df = awsmcs_wban_df[awsmcs_wban_df['NAME'].notna()]
    sl_data = pd.DataFrame({'AWSMSC':['720637'],'WBAN':['00223'],'NAME':'SUGAR LAND REGIONAL AIRPORT'})
    awsmcs_wban_df = pd.concat([awsmcs_wban_df, sl_data])
    awsmcs_wban_df['AWSMSC_WBAN'] = awsmcs_wban_df['AWSMSC'] + awsmcs_wban_df['WBAN']

    awsmcs_wban_df.to_sql(name='wban_stations', con=conn, if_exists='append', index=False)


### Houston Stations

In [10]:
hou_stations_df = awsmcs_wban_df[awsmcs_wban_df.NAME.str.contains('HOUSTON')].reset_index(drop=True)
hou_stations_df

,AWSMSC,WBAN,NAME,AWSMSC_WBAN
0,722429,53910,HOUSTON HOOKS MEMORIAL AP,72242953910
1,722430,12960,HOUSTON INTERCONTINENTAL AP,72243012960
2,722433,12969,HOUSTON LAKESIDE ARP,72243312969
3,722435,12918,HOUSTON WILLIAM P HOBBY AP,72243512918
4,722436,12906,HOUSTON ELLINGTON AFB,72243612906


## Data Download

In [20]:
def hrly_station_wx(awsmcs_wban, strt_dte, end_dte):
   '''Pulls hourly weather from NOAA Api'''

   try:
      os.makedirs(download_fldr)
   except FileExistsError:
      # directory already exists
      pass

   vars_dict = {'HourlyAltimeterSetting': 'float',
         'HourlyDewPointTemperature': 'int',
         'HourlyDryBulbTemperature': 'int',
         'HourlyPrecipitation': 'float',
         'HourlyPressureChange': 'float',
         'HourlyPressureTendency': 'int',
         'HourlyRelativeHumidity': 'int',
         'HourlySeaLevelPressure': 'float',
         'HourlyStationPressure': 'float',
         'HourlyVisibility': 'float',
         'HourlyWetBulbTemperature': 'int',
         'HourlyWindDirection': 'int',
         'HourlyWindGustSpeed': 'int',
         'HourlyWindSpeed': 'int'}

   hrly_vars = list(vars_dict.keys())

   hrly_vars_str = '&dataTypes='.join(hrly_vars)

   noaa_api_call = f'''https://www.ncei.noaa.gov/access/services/data/v1?dataset=local-climatological-data&dataTypes={hrly_vars_str}&stations={awsmcs_wban}&startDate={strt_dte}&endDate={end_dte}'''
   print(noaa_api_call)
   station_name = awsmcs_wban_df[awsmcs_wban_df.AWSMSC_WBAN==awsmcs_wban]['NAME'].iloc[0]

   filename = f'{station_name}_{strt_dte}_{end_dte}.csv'
   with open(os.path.join(download_fldr, filename), 'wb') as file:
      response = requests.get(noaa_api_call, allow_redirects=True)
      file.write(response.content)

   # get filepath and readin csv
   filepath = os.path.join(download_fldr,filename) 

   df = pd.read_csv(filepath)

   # column formatting
   df['DATE'] = pd.to_datetime(df.DATE)

   # get a list of all non-main columns that are also object datatypes
   # all should be numeric but sometimes they have random strings in them, so the str needs to be removed
   object_var_cols = list(set(df.select_dtypes(include=['object']).columns).intersection(set(hrly_vars)))

   # remove any strings from these variable columns because they shouldnt have any strings in them
   df[object_var_cols] = df[object_var_cols].replace(r'[^\d.]+', '',regex=True)

   #fill NaNs and change to ints
   df[hrly_vars] = df[hrly_vars].fillna(0)
   df[hrly_vars] = df[hrly_vars].replace('','0')
   df = df.astype(vars_dict)

   df.insert(1,'STATION_NAME', station_name)
   df.columns = df.columns.str.lower()

   # sort and keep the last
   df = df[df.report_type.isin(['FM-12','FM-15','FM-16'])].reset_index(drop=True)
   df['source'] = df.source.astype(int)
   df.sort_values(by='source', ascending=True, inplace=True)
   df.drop_duplicates(keep='last', subset=['station','date'],inplace=True)

   return df


x = pd.concat([hrly_station_wx(awsmcs_wban=i,strt_dte='2020-01-01',end_dte='2023-12-31') for i in hou_stations_df.AWSMSC_WBAN])
x = x[x.report_type.isin(['FM-12','FM-15','FM-16'])].reset_index(drop=True)
x['source'] = x.source.astype(int)
x.sort_values(by='source', ascending=True, inplace=True)

awsmcs_wban_df[awsmcs_wban_df.NAME.str.contains('SUGAR')]

In [21]:
sl =hrly_station_wx(awsmcs_wban='72063700223',strt_dte='1980-01-01',end_dte='2023-12-31')

https://www.ncei.noaa.gov/access/services/data/v1?dataset=local-climatological-data&dataTypes=HourlyAltimeterSetting&dataTypes=HourlyDewPointTemperature&dataTypes=HourlyDryBulbTemperature&dataTypes=HourlyPrecipitation&dataTypes=HourlyPressureChange&dataTypes=HourlyPressureTendency&dataTypes=HourlyRelativeHumidity&dataTypes=HourlySeaLevelPressure&dataTypes=HourlyStationPressure&dataTypes=HourlyVisibility&dataTypes=HourlyWetBulbTemperature&dataTypes=HourlyWindDirection&dataTypes=HourlyWindGustSpeed&dataTypes=HourlyWindSpeed&stations=72063700223&startDate=1980-01-01&endDate=2023-12-31


/tmp/ipykernel_940/2502798668.py:41: DtypeWarning: Columns (4,5,6,12,13,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [33]:
sql_create_hrly_wx = ''' Create Table if not exists hrly_wx (
                                    STATION text,
                                    STATION_NAME text,
                                    DATE text,
                                    REPORT_TYPE text,
                                    SOURCE integer,
                                    HOURLYALTIMETERSETTING real,
                                    HOURLYDEWPOINTTEMPERATURE integer,
                                    HOURLYDRYBULBTEMPERATURE integer,
                                    HOURLYPRECIPITATION real,
                                    HOURLYPRESSURECHANGE real,
                                    HOURLYPRESSURETENDENCY integer,
                                    HOURLYRELATIVEHUMIDITY integer,
                                    HOURLYSEALEVELPRESSURE real,
                                    HOURLYSTATIONPRESSURE real,
                                    HOURLYVISIBILITY real,
                                    HOURLYWETBULBTEMPERATURE integer,
                                    HOURLYWINDDIRECTION integer,
                                    HOURLYWINDGUSTSPEED integer,
                                    HOURLYWINDSPEED integer
                                    );
                                '''

bp.create_table(db_name, sql_create_hrly_wx)

# creates 
bp.create_table(db_name, 'CREATE UNIQUE INDEX if not exists ux_hrly_wban_wthr ON hrly_wx(STATION, STATION_NAME, DATE)')

In [5]:
# conn to db

db_name = 'weather.db'
conn = bp.create_connection(db_name)


In [28]:
bp.select_db_master_tbl(db_name)

    type               name       tbl_name  rootpage  \
0  table            hrly_wx        hrly_wx         2   
1  table      wban_stations  wban_stations         3   
2  index  ux_hrly_wban_wthr        hrly_wx        29   

                                                 sql  
0  CREATE TABLE hrly_wx (\n                      ...  
1  CREATE TABLE wban_stations (\n                ...  
2  CREATE UNIQUE INDEX ux_hrly_wban_wthr ON hrly_...  
